In [151]:
%pylab inline
%precision 6astype(float)

Populating the interactive namespace from numpy and matplotlib


'%r'

In [152]:
import re
import nltk
import csv
import pandas as pd
import numpy as np
import sklearn as skl
import math
import time
import datetime
from sklearn.preprocessing import StandardScaler

## Parser

In [153]:
f = open("train.csv", encoding="cp1251")
f.readline()

'PostId,PostCreationDate,OwnerUserId,OwnerCreationDate,ReputationAtPostCreation,OwnerUndeletedAnswerCountAtPostTime,Title,BodyMarkdown,Tag1,Tag2,Tag3,Tag4,Tag5,PostClosedDate,OpenStatus\n'

In [154]:
real_test_path="test.csv"
datarealtest = pd.read_csv(real_test_path, encoding="cp1251")

In [155]:
class Data:
    def __init__(self, lst):
        self.postId = lst[0]
        self.postCreationDate = lst[1]
        self.ownerUserId = lst[2]
        self.ownerCreationDate = lst[3]
        self.reputationAtPostCreation = lst[4]
        self.ownerUndeletedAnswerCountAtPostTime = lst[5]
        self.title = lst[6]
        self.bodyMarkdown = lst[7]
        self.tags = lst[8:13]
        self.postClosedDate = lst[13]
        self.openStatus = lst[14]

    def getList(self):
        res = []
        res.append(self.postId)
        res.append(self.postCreationDate)
        res.append(self.ownerUserId)
        res.append(self.ownerCreationDate)
        res.append(self.reputationAtPostCreation)
        res.append(self.ownerUndeletedAnswerCountAtPostTime)
        res.append(self.title)
        res.append(self.bodyMarkdown)
        for i in self.tags:
            if len(i)==0:
                res.append("NaN")
            else:
                res.append(i)
        
        if len(self.postClosedDate)==0:
            res.append("0")
        else:
            res.append(self.postClosedDate)
        
        d='01234'
        if len(self.openStatus)==0 or not self.openStatus in d:
            res="NaN"
        else:
            res.append(self.openStatus)
        
        return res

    def __unicode__(self):
        return "{id: %s}" % self.postId

    def __str__(self):
        return self.__unicode__()

In [156]:
def readOne():
    regexp = r'(?<!")"(?!")'
    fullStr = r'",([a-zA-Z0-9!@#$%^&+*\(\)\[\]\\\/.-]*,){5}(\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2})*,\d'

    s = f.readline()
    if len(s) == 0:
        return []
    params = s.strip().split(",")

    toTest = '",' + ",".join(params[-7:])
    if re.match(fullStr, toTest):
        # wow, we have a single-line data, good
        return params

    body = "".join(params[7:])[1:]
    params = params[:8]
    while True:
        s = f.readline()
        match = re.search(fullStr, s)
        if match:
            # found end quote, parsing data left
            offs = match.start(0)
            body += s[:offs]
            left = s[offs + 2:]
            params[7] = body
            params += left.strip().split(",")
            return params

        body += s

In [158]:
res = []

for i in range(5000):
#while True:
    src = readOne()
    
    if (len(src) == 0):
        break
    res.append(Data(src))

In [159]:
print (len(res))

5000


In [160]:
cols = ['PostId', 'PostCreationDate', 'OwnerUserId', 'OwnerCreationDate', 'ReputationAtPostCreation', 
           'OwnerUndeletedAnswerCountAtPostTime', 'Title', 'BodyMarkdown', 'Tag1', 'Tag2', 'Tag3', 'Tag4', 
           'Tag5', 'PostClosedDate', 'OpenStatus']
listData = []
for i in range(len(res)):
    l = res[i].getList()
    if l!="NaN":
        listData.append(l)

In [161]:
data = pd.DataFrame(listData, columns=cols)

In [162]:
data.OpenStatus.value_counts()

0    3245
4     405
2     381
3     353
1     343
Name: OpenStatus, dtype: int64

In [163]:
data.ReputationAtPostCreation=data.ReputationAtPostCreation.astype(int)
data.PostId=data.PostId.astype(int)
data.OwnerUserId=data.OwnerUserId.astype(int)
data.OwnerUndeletedAnswerCountAtPostTime=data.OwnerUndeletedAnswerCountAtPostTime.astype(int)
data.OwnerUserId=data.OwnerUserId.astype(int)

In [108]:
datarealtest.ReputationAtPostCreation=data.ReputationAtPostCreation.astype(int)
datarealtest.PostId=data.PostId.astype(int)
datarealtest.OwnerUserId=data.OwnerUserId.astype(int)
datarealtest.OwnerUndeletedAnswerCountAtPostTime=data.OwnerUndeletedAnswerCountAtPostTime.astype(int)
datarealtest.OwnerUserId=data.OwnerUserId.astype(int)

## Make new features

In [164]:
def find_code(data):
    symbols = ['$','(', '{', '[', '=', '<', '#', '%', '@']

    textCol=[]
    codeCol=[]
    for index, row in data.iterrows():
        words=[]
        words = str(row.BodyMarkdown)
        sents = words.split('\n\n\n')
        text=''
        code=''
        for sent in sents:
            cnt = 0
            for sym in symbols:
                cnt = cnt + sent.count(sym)
            if cnt>3:
                code = code +sent+' '
            else:
                text = text + sent + ' '
        
        textCol.append(text)
        codeCol.append(code)
        
    data['Text']=textCol
    data['Code'] = codeCol


In [165]:
find_code(data)

In [111]:
find_code(datarealtest)

In [166]:
def deleteBags(data):
    bags = ['\n','\t','\\n','(','"','+','-','=',',',';',')','{',
            '}','[',']','?','!','$', '|', '<', '>', '#','\\0','*']
    for s in bags:
        data.Text = data.Text.apply(lambda x: x.replace(s,''))
        data.Code = data.Code.apply(lambda x: x.replace(s,''))


In [167]:
deleteBags(data)

In [114]:
deleteBags(datarealtest)

In [169]:
import nltk
nltk.download('punkt')

# Do this in your ipython notebook or analysis script
from nltk.tokenize import word_tokenize

def get_time(x):
    try:
        return time.mktime(datetime.datetime.strptime(x, '%m/%d/%Y %H:%M:%S').timetuple())
    except:
        return time.mktime(datetime.datetime.strptime(x, '%Y-%m-%d').timetuple())
    
def make_new_features(data):
    ls = []
    tg = []
    dt=[]
    for ind,row in data.iterrows():        
        s = str(row.Tag1) + ' ' + str(row.Tag2) + ' ' + str(row.Tag3) + ' ' + str(row.Tag4) + ' ' + str(row.Tag5)
        if s.find("NaN")!=-1:
            s = s[0:s.find("NaN")]
        tg.append(len(s.split(' '))-1)
        ls.append(s)

    #data['PostClosedDif'] = dt
    data['Tags'] = ls
    data['TagsNumber']= tg
    data['TitleWordsCnt'] = data.Title.apply(lambda x: len(nltk.word_tokenize(re.sub('\W+', ' ', x.lower()))))

    data['BodyTextCnt'] = data.Text.apply(lambda x: len(nltk.word_tokenize(re.sub('\W+', ' ', x.lower()))))
    data['BodyCodeCnt'] = data.Code.apply(lambda x: len(nltk.word_tokenize(re.sub('\W+', ' ', x.lower()))))
    data['BodyLength'] = data.BodyMarkdown.apply(lambda x: len(x.split()))
    data['TitleLength'] = data.Title.apply(lambda x: len(x.split()))
    
    data['BodySentanceCnt'] = data.BodyMarkdown.apply(lambda x: len(nltk.sent_tokenize(x.lower())))
    data['PostCreation'] = data.PostCreationDate.apply(lambda x: get_time(x))
    data['OwnerCreation'] = data.OwnerCreationDate.apply(lambda x: get_time(x))
    data['PostAge'] = data.PostCreation - data.OwnerCreation
 
    #data['BodyEntersCnt']= data.BodyMarkdown.apply(lambda x: x.count('\n') )
    br = data.BodyMarkdown.apply(lambda x: x.count('{')) + data.BodyMarkdown.apply(lambda x: x.count('('))
    br = br + data.BodyMarkdown.apply(lambda x: x.count('<'))
    data['BodyBraceCnt'] = br
    data['BodyLinksCnt'] = data.BodyMarkdown.apply(lambda x: x.count('://') )
    #data['BodyDogCount'] = data.BodyMarkdown.apply(lambda x: x.count('@'))
    #data['BodyEqCount'] = data.BodyMarkdown.apply(lambda x: x.count('='))
    #data['Title?Cnt'] = data.Title.apply(lambda x: x.count('?'))
    #data['TitleSpaceCnt']=data.Title.apply(lambda x: x.count(' '))
    data['PostAge'] = data.PostAge.apply(lambda x: max(x, 1))



[nltk_data] Downloading package punkt to /home/argali/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [170]:
make_new_features(data)

In [ ]:
make_new_features(datarealtest)

## Split data

In [198]:
n = 2500
data_train=data[:n]
n2 = len(data)-n
#data_train = data
data_test = data[-n2:]
#data_real_test = datarealtest
features = ['ReputationAtPostCreation', 'OwnerUndeletedAnswerCountAtPostTime', 'TagsNumber',
            'TitleWordsCnt', 'BodyTextCnt', 'BodySentanceCnt', 
            'PostAge','BodyLinksCnt','BodyBraceCnt', 'BodyCodeCnt'
            #'PostClosedDif'
            #'TextLength', 
            #'TitleLength','BodyEntersCnt','BodyBracketCnt','BodyAngleCnt','BodyDogCount','BodyEqCount'
            #'Title?Cnt', 'TitleSpaceCnt',
           ]
X_train = np.array(data_train[features])
X_test = np.array(data_test[features])
#X_real_test = np.array(data_real_test[features])
y_train = np.array(data_train.OpenStatus)
y_test = np.array(data_test.OpenStatus)


In [199]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [200]:
X_real_test = scaler.transform(X_real_test)

NameError: name 'X_real_test' is not defined

## Tfidf

In [203]:
docs = data_train.Text
tags = data_train.Tags + data_train.Title
title = data_train.Title

In [204]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(min_df=50, analyzer='word', ngram_range=(1, 2))
vect.fit(docs)
X_train_tfidf = vect.transform(data_train.BodyMarkdown)
X_test_tfidf = vect.transform(data_test.BodyMarkdown)

In [205]:
vect_tags = TfidfVectorizer(min_df=50, stop_words='english', analyzer='word', ngram_range=(1, 2))
vect_tags.fit(tags)
X_train_tfidf_tags = vect.transform(data_train.BodyMarkdown)
X_test_tfidf_tags = vect.transform(data_test.BodyMarkdown)

In [206]:
X_train_tfidf.shape

(2500, 604)

In [207]:
import scipy.sparse as sps
X_train_2 = sps.hstack((X_train, X_train_tfidf))
X_test_2 = sps.hstack((X_test, X_test_tfidf))

In [208]:
X_train_2 = sps.hstack((X_train_2, X_train_tfidf_tags))
X_test_2 = sps.hstack((X_test_2, X_test_tfidf_tags))

## Classifier

Just test on KNN

In [179]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

clf = KNeighborsClassifier(n_neighbors=10)
clf.fit(X_train, y_train)
y_hat = clf.predict(X_test)
skl.metrics.accuracy_score(y_test, y_hat)

0.68720000000000003

In [201]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=500, criterion = 'gini')
clf.fit(X_train, y_train)
y_hat = clf.predict(X_test)
skl.metrics.accuracy_score(y_test, y_hat)

0.69241131567130665

In [181]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier()
clf.fit(X_train, y_train)
y_hat = clf.predict(X_test)
skl.metrics.accuracy_score(y_test, y_hat)

0.6976

## Linear

In [216]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

clf_linear = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=0.3, 
                                intercept_scaling=1, random_state=1, 
                                solver='lbfgs', 
                                max_iter=50, multi_class='multinomial', verbose=0.5)
clf_linear.fit(X_train_2, y_train)
y_hat = clf_linear.predict(X_test_2)
skl.metrics.accuracy_score(y_test, y_hat)
np.mean(y_test == y_hat)   

/home/argali/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:701: UserWarning: lbfgs failed to converge. Increase the number of iterations.
  warnings.warn("lbfgs failed to converge. Increase the number "
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s finished


0.71576111360574768

## Result

In [ ]:
res = datarealtest.copy()
res['OpenStatus'] = ans

In [ ]:
res.to_csv('result.csv', mode='w', index=False, columns=['PostId', 'OpenStatus'])